In [2]:
%%time
print("starting")
#import in all libraries and methods
# %run -i 'myimports.py'
# %run -i 'datagathering.py'
# %run -i 'nlp_processing.py' #imports packages and methods
from checkra import happyscribe_scrape as happy

main = "https://www.happyscribe.com"                
url="/public/lex-fridman-podcast-artificial-intelligence-ai"
podcast_host = "Lex Fridman"
html_location = "Archive/happyscribe_Lex_Fridman_html.txt"

happy.update_transcripts(html_location, main)

starting
CPU times: user 13 s, sys: 628 ms, total: 13.6 s
Wall time: 25.9 s


In [1]:
%%time
%run -i 'myimports.py'
%run -i 'nlp_processing.py' #imports packages and methods


doc = nlp(text_clean.text_fix(open("3Lex/#107|Peter_Singer|Suffering_in_Humans,_Animals,_and_AI.txt").read()))

print(doc.user_data.keys())


NameError: name 'checkra' is not defined

In [2]:
%%time
doc = nlp(text_fix(open("3Lex/#101|Joscha_Bach|Artificial_Consciousness_and_the_Nature_of_Reality.txt").read()))
# doc = nlp(text_fix(open("3Lex/#103|Ben_Goertzel|Artificial_General_Intelligence.txt").read()))

NameError: name 'get_algo_timestamps' is not defined

In [3]:
print(doc.user_data["traits"].keys())

dict_keys(['People', 'Books', 'Topics', 'Places', 'Products/Companies', 'Events', 'Laws', 'Identity Groups', 'All Mentions'])


In [2]:
%%time
print("starting")
folder="3Lex"
# docs, doc_bin = process_folder_to_docs("5Lex", podcast_host)
onlyfiles=[(text_fix(open(folder+"/"+file).read()), file) for file in listdir(folder) if isfile(join(folder, file))]
docs = [] #list of docs 

for doc, name in tqdm(nlp.pipe(onlyfiles, as_tuples=True)): #piping all collection of docs to make doclist and docbin
    #each doc contains hostname, guest, title, entities mentioned, and summary
    name = re.split("[\|]",name)
    name=name[1:] if name[0][1:].isdigit() else name # store name of guest and topic, add to doc user data
    
    doc.user_data["host"] = podcast_host
    doc.user_data["guest"]= str(name[0]).replace("_"," ")
    doc.user_data["title"]= str(name[1][:-4]).replace("_"," ")
    doc = subtopics(doc)
    docs.append(doc)

print("done")


starting


0it [00:00, ?it/s]

The following is a conversation with Ben Gursel, one of the most interesting minds in the artificial intelligence community.
he is the founder of Singularity, not designer of Open Kagi Framework, formerly a director of research at the Machine Intelligence Research Institute and chief scientist of HANDSON Robotics, the company that created the Sophea robot.
He has been a central figure in the Ajai community for many years, including in his organizing and contributing to the conference and artificial general intelligence, the 2020 version of which is actually happening this week, Wednesday, Thursday and Friday.


4it [04:01, 74.94s/it] /ilab/users/hs884/.local/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /ilab/users/hs884/.local/lib/python3.8/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
31it [14:10, 23.44s/it]

The following is a conversation with Stephen Wolfram, his second time in the podcast.
he is a computer scientist, mathematician, theoretical physicist and the founder and CEO of Wolfram Research, a company behind Mathematica, Wolfram Alpha, Wolfram Language and the new Wolfram Physics Project.
he is the author of several books, including A New Kind of Science and the new book, A Project to Find the Fundamental Theory of Physics.


46it [20:27, 26.69s/it]

done
CPU times: user 10min 51s, sys: 7min 34s, total: 18min 25s
Wall time: 20min 30s


In [5]:
doc.user_data["traits"]["Events"]

['AI Winters']

In [6]:
print(docs[0].user_data["guest"])

Ben Goertzel


In [4]:
for ind, doc in enumerate(docs):
    print(ind, doc.user_data["guest"], doc.user_data["title"])

0 Ben Goertzel Artificial General Intelligence
1 Steven Pressfield The War of Art
2 Alexander Fridman My Dad, the Plasma Physicist
3 Robert Langer Edison of Medicine
4 Joscha Bach Artificial Consciousness and the Nature of Reality
5 Yaron Brook Ayn Rand and the Philosophy of Objectivism
6 John Clarke The Art of Fighting and the Pursuit of Excellence
7 David Patterson Computer Architecture and Data Storage
8 Matt Botvinick Neuroscience, Psychology, and AI at DeepMind
9 Matthew Johnson Psychedelics
10 Andrew Huberman Neuroscience of Optimal Performance
11 Peter Singer Suffering in Humans, Animals, and AI
12 Sergey Levine Robotics and Machine Learning
13 Brian Kernighan UNIX, C, AWK, AMPL, and Go Programming
14 Jitendra Malik Computer Vision
15 Ian Hutchinson Nuclear Fusion, Plasma Physics, and Religion
16 Lisa Feldman Barrett Love, Evolution, and the Human Brain
17 Manolis Kellis Human Genome and Evolutionary Dynamics
18 Erik Brynjolfsson Economics of AI, Social Networks, and Technology


In [4]:
docs[0].user_data["traits"].keys()

dict_keys(['People', 'Books', 'Topics', 'Places', 'Products/Companies', 'Events', 'Laws', 'Identity Groups'])

In [7]:
import shutil
count = 0
not_processed = []
for ind, file in enumerate(os.listdir("7Lex")):
    if file.endswith(".txt"):
        file2 = file.replace("_", " ").replace("|", " ").replace(".txt", " ")
    #     print(ind, file) 
        processed = False


        for doc in docs:
            if doc.user_data["guest"] in file2 and doc.user_data["title"] in file2:
    #             os.rename("5Lex/"+file, "7Lex/"+file)
                
                processed = True
                count+=1
                break
        if processed:
            os.rename("7Lex/"+file, "5Lex/"+file)
print("done")

done


In [14]:
%%time
def att_to_csv(docs, atts):
    all_atts= sorted(set([item for doc in docs for item in doc.user_data[atts]])) #all atts
    all_attributes=dict([(x, [x]) for x in all_atts]) #dictionary of books to become key is base book, value are similar titles
    i = 0
#     similar_words implement dictionary to store similar words that were deleted
    while i<len(all_attributes)-1: #remove similar or subwords
        str1, str2 = all_atts[i], all_atts[i+1]
        if str2 in str1 or (SequenceMatcher(a=str1,b=str2).ratio()>.8 and len(str1)>len(str2)):
            toRemove = all_attributes.get(str2) #list of similar words moving
            toKeep =  all_attributes.get(str1)
            
            try:
                del all_attributes[str1]
            except:
                pass
            all_attributes[str1] = toRemove+toKeep
        elif str1 in str2 or (SequenceMatcher(a=str1,b=str2).ratio()>.8 and len(str1)<=len(str2)):
            toRemove = all_attributes.get(str1) #list of similar words moving
            toKeep =  all_attributes.get(str2)
            try:
                del all_attributes[str2]
            except:
                pass
            all_attributes[str2] = toRemove+toKeep
            
        i+=1
    new_dic = {} #reverse dict so that given any name, can find base
    for k,v in all_attributes.items():
        for x in v:
            new_dic.setdefault(x,[]).append(k)
    #now have dict of base books with values of similar titles
        
    all_attributes_dict = dict(zip(all_attributes.keys(), np.arange(len(all_attributes)))) #create dict for all entities
    
    
    guests = [doc.user_data["guest"] for doc in docs]
    guests_dict = dict(zip(guests, np.arange(len(all_attributes), len(all_attributes)+len(guests))))#create dict for all main
    
#     #now create dict for all edges by going from every doc's mentions of attribute, and adding edge from guest to attribute
    
    i = len(guests) + len(all_attributes) #make keys for final edges

    edges = []
    for doc in docs:
        current_name = guests_dict.get(doc.user_data["guest"]) #get graph id for each guest in doc
        for mention in doc.user_data[atts]:
            if new_dic.get(mention):
#                 print(new_dic.get(mention))
                edges.append((current_name, all_attributes_dict.get(new_dic.get(mention)[0]))) #from speaker to mention of base book
    edges_dict=dict(zip(edges, np.arange(i, i+len(edges))))
    
    return all_attributes_dict, guests_dict, edges_dict

all_attributes_dict, guests_dict, edges_dict = att_to_csv(docs1, "people")

CPU times: user 20.2 ms, sys: 0 ns, total: 20.2 ms
Wall time: 19.5 ms


In [10]:
docs1[0].user_data["guest"]

'Alexander Fridman'

In [ ]:
all_attributes=dict([(x, [x]) for x in all_atts]) #dictionary of books to become key is base book, value are similar titles
i = 0
#     similar_words implement dictionary to store similar words that were deleted
while i<len(all_attributes)-1: #remove similar or subwords
    str1, str2 = all_atts[i], all_atts[i+1]
    if str2 in str1 or (SequenceMatcher(a=str1,b=str2).ratio()>.8 and len(str1)>len(str2)):
        print("1", str1, str2)
        toRemove = all_attributes.get(str2) #list of similar words moving
        toKeep =  all_attributes.get(str1)

        try:
            print(all_attributes.keys(),"\n")
            del all_attributes[str1]
            print(all_attributes.keys())
        except:
            print("rip")
            pass
        all_attributes[str1] = toRemove+toKeep
    elif str1 in str2 or (SequenceMatcher(a=str1,b=str2).ratio()>.8 and len(str1)<=len(str2)):
        print("2",str1, str2)
        toRemove = all_attributes.get(str1) #list of similar words moving
        toKeep =  all_attributes.get(str2)
        try:
            del all_attributes[str2]
        except:
            pass
        all_attributes[str2] = toRemove+toKeep

    i+=1

In [8]:
username = urllib.parse.quote_plus('Empyr3an')
password = urllib.parse.quote_plus("B4ldr1c7@1")

cluster = pymongo.MongoClient("mongodb+srv://{}:{}@cluster0.4pec2.mongodb.net/Checkra?retryWrites=true&w=majority".format(username, password))
db = cluster["production"]
collection = db["lex"]

# db.create_collection("lmini")

In [38]:
queried = list(db["lex"].find({},{"_id":0,"traits.Events":1}))
all_ents = [ent for doc in queried for ent in doc["traits"]["Events"]]

In [9]:
db["lex"].drop()
db["lex"].insert_many([doc.user_data for doc in docs])

In [ ]:
all_atts = []
for doc in list(collection.find({"books":{"$ne":[]}})):
    print(doc["guest"], doc[att])
    for a in doc[att]:
        all_atts.append(a)

In [2]:
%%time
#loading in previous bytestreams, needs 2 since 2 podcast folders edit later
doc_bin = DocBin(store_user_data=True) #docbin container for serialization

    doc_bin.add(doc) #add doc to list and bin
    
# with open("cucumber/Lexnumbered.bin", "wb") as f: #write bytestream to first_doc.bin
#     f.write(doc_bin.to_bytes())

with open("cucumber/Lex.bin", "rb") as w: 
    new_docs1 = DocBin(store_user_data=True).from_bytes(w.read())
docs1 = list(new_docs1.get_docs(nlp.vocab))

Love, Evolution, and the Human Brain Lisa Feldman Barrett
Quantum Computing Scott Aaronson
Economics of AI, Social Networks, and Technology Erik Brynjolfsson
Microsoft CTO Kevin Scott
Psychedelics Matthew Johnson
Reinforcement Learning and the Future of AI Michael Littman
Deep Learning, Education, and Real-World AI Andrew Ng
The Hard Problem of Consciousness David Chalmers
Cruise Automation Kyle Vogt
Thinking Fast and Slow, Deep Learning, and AI Daniel Kahneman
Effective Altruism William MacAskill
IBM Watson, Jeopardy & Deep Conversations with AI David Ferrucci
Space Exploration, Space Suits, and Life on Mars Dava Newman
Algorithms, TeX, Life, and The Art of Computer Programming Donald Knuth
Neuralink, AI, Autopilot, and the Pale Blue Dot Elon Musk
Tesla Autopilot Elon Musk
The Art of Fighting and the Pursuit of Excellence John Clarke
Universal Artificial Intelligence, AIXI, and AGI Marcus Hutter
Physics View of the Mind and Neurobiology John Hopfield
Stack Overflow and Coding Horror J

In [ ]:
att_to_csv(docs1, "people") #to get csv of edges

#generate megagraph for book ents
edges = pd.read_csv("booksEdges.csv", sep=',').drop("id", axis=1).values.tolist()
nodes = pd.read_csv('booksNodes.csv',sep=',').set_index("id").to_dict("index")

source = [nodes.get(e[0])["name"] for e in edges]
target = [nodes.get(e[1])["name"] for e in edges]
kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':np.ones(len(source))})
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())
plt.figure(figsize=(12,12))

pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

#following are other example graphviews

In [ ]:
G=nx.from_pandas_edgelist(kg_df[kg_df['target']=="Bible"], "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regulates the distance between nodes



nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()